In [1]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

In [65]:
import keras
from keras import layers
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

In [87]:
# Build the encoder and decoder

latent_dim = 14     # dimensionality of the compressed latent space
n_indicators = 14  # number of input features

encoder_in = keras.Input(shape=(n_indicators,))
hidden1 = layers.Dense(14, activation='relu')(encoder_in)
hidden_encoded = layers.Dense(latent_dim, activation='relu')(hidden1)
encoder = keras.Model(encoder_in, hidden_encoded, name='encoder')
encoder.summary()

decoder_in = keras.Input(shape=(latent_dim,))
decode_hidden1 = layers.Dense(14, activation='sigmoid')(decoder_in)
decoded_layer = layers.Dense(n_indicators, activation='linear')(decode_hidden1)
decoder = keras.Model(decoder_in, decoded_layer, name='decoder')
decoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 14)]              0         
_________________________________________________________________
dense_58 (Dense)             (None, 14)                210       
_________________________________________________________________
dense_59 (Dense)             (None, 14)                210       
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, 14)]              0         
_________________________________________________________________
dense_60 (Dense)             (None, 14)                210       
____________________________________________

In [26]:
# Define the autoencoder model
class Autoencoder(keras.Model):
    
    def __init__(self, encoder, decoder, **kwargs):
        super(Autoencoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        
    @property
    def metrics(self):
        return [self.loss_tracker]
        
    def call(self, inputs):
        compressed_input = self.encoder(inputs)
        out = self.decoder(compressed_input)
        return out
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )

        grads = tape.gradient(reconstruction_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.loss_tracker.update_state(reconstruction_loss)
        return {"loss": self.loss_tracker.result()}

In [88]:
data = pd.read_csv(f'./data/TM.csv')
# generate matrix with technical indicators (features) as columns
features = data.to_numpy()[:, 6:]
scaler = StandardScaler()
X = scaler.fit_transform(features)

print(X.shape)

feats_train = tf.convert_to_tensor(X[:2175, :], dtype=tf.float64)
feats_test = tf.convert_to_tensor(X[2175:, :], dtype=tf.float64)

# ae = Autoencoder(encoder, decoder)
# ae.compile(optimizer=keras.optimizers.Adam())
# ae.fit(feats_train, epochs=100, batch_size=32)

autoencoder = keras.Model(encoder_in, decoder(encoder(encoder_in)), name="autoencoder")
autoencoder.summary()

autoencoder.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.MeanSquaredError())
autoencoder.fit(feats_train, feats_train,
                epochs=500,
                batch_size=16,
                shuffle=True,
                validation_data=(feats_test, feats_test))

(2417, 14)
Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 14)]              0         
_________________________________________________________________
encoder (Functional)         (None, 14)                420       
_________________________________________________________________
decoder (Functional)         (None, 14)                420       
Total params: 840
Trainable params: 840
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
136/136 [==============================] - 1s 2ms/step - loss: 1.0611 - val_loss: 0.9327
Epoch 2/500
136/136 [==============================] - 0s 777us/step - loss: 0.7168 - val_loss: 0.6554
Epoch 3/500
136/136 [==============================] - 0s 798us/step - loss: 0.4498 - val_loss: 0.5149
Epoch 4/500
136/136 [==============================] - 0s 849us/step 

136/136 [==============================] - 0s 752us/step - loss: 0.0317 - val_loss: 0.0513
Epoch 73/500
136/136 [==============================] - 0s 759us/step - loss: 0.0280 - val_loss: 0.0497
Epoch 74/500
136/136 [==============================] - 0s 749us/step - loss: 0.0270 - val_loss: 0.0497
Epoch 75/500
136/136 [==============================] - 0s 737us/step - loss: 0.0288 - val_loss: 0.0486
Epoch 76/500
136/136 [==============================] - 0s 761us/step - loss: 0.0281 - val_loss: 0.0482
Epoch 77/500
136/136 [==============================] - 0s 753us/step - loss: 0.0281 - val_loss: 0.0457
Epoch 78/500
136/136 [==============================] - 0s 747us/step - loss: 0.0272 - val_loss: 0.0464
Epoch 79/500
136/136 [==============================] - 0s 769us/step - loss: 0.0269 - val_loss: 0.0452
Epoch 80/500
136/136 [==============================] - 0s 759us/step - loss: 0.0266 - val_loss: 0.0441
Epoch 81/500
136/136 [==============================] - 0s 770us/step - loss:

136/136 [==============================] - 0s 815us/step - loss: 0.0189 - val_loss: 0.0326
Epoch 151/500
136/136 [==============================] - 0s 777us/step - loss: 0.0184 - val_loss: 0.0326
Epoch 152/500
136/136 [==============================] - 0s 778us/step - loss: 0.0185 - val_loss: 0.0318
Epoch 153/500
136/136 [==============================] - 0s 786us/step - loss: 0.0187 - val_loss: 0.0323
Epoch 154/500
136/136 [==============================] - 0s 801us/step - loss: 0.0188 - val_loss: 0.0328
Epoch 155/500
136/136 [==============================] - 0s 781us/step - loss: 0.0177 - val_loss: 0.0348
Epoch 156/500
136/136 [==============================] - 0s 753us/step - loss: 0.0193 - val_loss: 0.0323
Epoch 157/500
136/136 [==============================] - 0s 794us/step - loss: 0.0192 - val_loss: 0.0313
Epoch 158/500
136/136 [==============================] - 0s 781us/step - loss: 0.0186 - val_loss: 0.0323
Epoch 159/500
136/136 [==============================] - 0s 800us/ste

136/136 [==============================] - 0s 820us/step - loss: 0.0121 - val_loss: 0.0225
Epoch 229/500
136/136 [==============================] - 0s 777us/step - loss: 0.0131 - val_loss: 0.0200
Epoch 230/500
136/136 [==============================] - 0s 762us/step - loss: 0.0125 - val_loss: 0.0193
Epoch 231/500
136/136 [==============================] - 0s 757us/step - loss: 0.0120 - val_loss: 0.0198
Epoch 232/500
136/136 [==============================] - 0s 761us/step - loss: 0.0122 - val_loss: 0.0188
Epoch 233/500
136/136 [==============================] - 0s 775us/step - loss: 0.0125 - val_loss: 0.0202
Epoch 234/500
136/136 [==============================] - 0s 782us/step - loss: 0.0120 - val_loss: 0.0187
Epoch 235/500
136/136 [==============================] - 0s 785us/step - loss: 0.0115 - val_loss: 0.0197
Epoch 236/500
136/136 [==============================] - 0s 765us/step - loss: 0.0123 - val_loss: 0.0185
Epoch 237/500
136/136 [==============================] - 0s 1ms/step 

136/136 [==============================] - 0s 793us/step - loss: 0.0104 - val_loss: 0.0163
Epoch 307/500
136/136 [==============================] - 0s 772us/step - loss: 0.0106 - val_loss: 0.0168
Epoch 308/500
136/136 [==============================] - 0s 758us/step - loss: 0.0101 - val_loss: 0.0178
Epoch 309/500
136/136 [==============================] - 0s 835us/step - loss: 0.0108 - val_loss: 0.0169
Epoch 310/500
136/136 [==============================] - 0s 791us/step - loss: 0.0106 - val_loss: 0.0170
Epoch 311/500
136/136 [==============================] - 0s 798us/step - loss: 0.0100 - val_loss: 0.0181
Epoch 312/500
136/136 [==============================] - 0s 783us/step - loss: 0.0105 - val_loss: 0.0179
Epoch 313/500
136/136 [==============================] - 0s 799us/step - loss: 0.0105 - val_loss: 0.0170
Epoch 314/500
136/136 [==============================] - 0s 773us/step - loss: 0.0102 - val_loss: 0.0162
Epoch 315/500
136/136 [==============================] - 0s 786us/ste

136/136 [==============================] - 0s 818us/step - loss: 0.0096 - val_loss: 0.0123
Epoch 385/500
136/136 [==============================] - 0s 804us/step - loss: 0.0089 - val_loss: 0.0118
Epoch 386/500
136/136 [==============================] - 0s 780us/step - loss: 0.0091 - val_loss: 0.0123
Epoch 387/500
136/136 [==============================] - 0s 782us/step - loss: 0.0095 - val_loss: 0.0117
Epoch 388/500
136/136 [==============================] - 0s 790us/step - loss: 0.0091 - val_loss: 0.0126
Epoch 389/500
136/136 [==============================] - 0s 797us/step - loss: 0.0087 - val_loss: 0.0121
Epoch 390/500
136/136 [==============================] - 0s 800us/step - loss: 0.0089 - val_loss: 0.0123
Epoch 391/500
136/136 [==============================] - 0s 792us/step - loss: 0.0090 - val_loss: 0.0121
Epoch 392/500
136/136 [==============================] - 0s 801us/step - loss: 0.0090 - val_loss: 0.0115
Epoch 393/500
136/136 [==============================] - 0s 782us/ste

136/136 [==============================] - 0s 779us/step - loss: 0.0083 - val_loss: 0.0117
Epoch 463/500
136/136 [==============================] - 0s 766us/step - loss: 0.0081 - val_loss: 0.0132
Epoch 464/500
136/136 [==============================] - 0s 805us/step - loss: 0.0085 - val_loss: 0.0129
Epoch 465/500
136/136 [==============================] - 0s 780us/step - loss: 0.0081 - val_loss: 0.0123
Epoch 466/500
136/136 [==============================] - 0s 800us/step - loss: 0.0078 - val_loss: 0.0112
Epoch 467/500
136/136 [==============================] - 0s 753us/step - loss: 0.0082 - val_loss: 0.0130
Epoch 468/500
136/136 [==============================] - 0s 765us/step - loss: 0.0084 - val_loss: 0.0118
Epoch 469/500
136/136 [==============================] - 0s 771us/step - loss: 0.0083 - val_loss: 0.0115
Epoch 470/500
136/136 [==============================] - 0s 819us/step - loss: 0.0082 - val_loss: 0.0121
Epoch 471/500
136/136 [==============================] - 0s 817us/ste

In [7]:
import keras
from keras import layers

latent_dim = 3 # size of the latent space


keras.input

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

%matplotlib inline

for ticker in tickers:
    # read in data from csv
    data = pd.read_csv(f'./data/{ticker}.csv')

    # generate matrix with technical indicators (features) as columns
    features = data.to_numpy()[:, 6:]
    
    


